# Adversarial example with neural network classifier
In this example, we redo the [adversarial example](https://gurobi-optimization-ml2gurobi.readthedocs-hosted.com/en/latest/examples/adversarial_mnist.html) of the documentation but use a logistic regression model for the regression

We don't detail the optimization model here. Please refer to the example in the documentation.

## Import the necessary packages and load data

We import all the package we need for this example.
We fetch the MINST data set using sklearn's functionalities.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import gurobipy as gp

In [ ]:
import sys

sys.path.append("../../src")

In [ ]:
from gurobi_ml import add_predictor_constr

In [ ]:
# Get MNIST digit recognition data set
# Load data from https://www.openml.org/d/554
X, Y = fetch_openml(
    "mnist_784", version=1, return_X_y=True, as_frame=False, parser="pandas"
)

Flatten and scale the data

In [ ]:
X = X.astype(np.float64) / 255.0
clf = MLPClassifier([20, 20])
pipeline = make_pipeline(clf)
pipeline.fit(X, Y)

In [ ]:
imageno = 11

In [ ]:
ex_prob = clf.predict_proba(X[imageno : imageno + 1, :])
sorted_labels = np.argsort(ex_prob)[0]
right_label = sorted_labels[-1]
wrong_label = sorted_labels[-2]

In [ ]:
wrong_label

In [ ]:
image = X[imageno, :]

In [ ]:
plt.imshow(image.reshape((28, 28)), cmap="gray")
label = clf.predict(image.reshape(1, -1))
print(f"Solution is classified as {label}")

In [ ]:
m = gp.Model()
delta = 5

x = m.addMVar(image.shape, lb=0.0, ub=1.0, name="x")
y = m.addMVar(ex_prob.shape, lb=-100, ub=100, name="y")

m.setObjective(y[0, wrong_label] - y[0, right_label], gp.GRB.MAXIMIZE)

# Bound on the distance to example in norm-1
abs_diff = m.addMVar(image.shape, lb=0, ub=1.0, name="abs_diff")
m.addConstr(abs_diff >= x - image)
m.addConstr(abs_diff >= image - x)
m.addConstr(abs_diff.sum() <= delta)

# Gurobi ML magic
pred_constr = add_predictor_constr(m, clf, x, y, out_activation="softmax")

pred_constr.print_stats()

In [ ]:
m.Params.OBBT = 3
m.optimize()

In [ ]:
plt.imshow(x.X.reshape((28, 28)), cmap="gray")
label = pipeline.predict(x.X.reshape(1, -1))
print(f"Solution is classified as {label}")

copyright © 2023 Gurobi Optimization, LLC